In [156]:
import pandas as pd
import numpy as np

In [105]:
filename = "/Users/lmatayoshi/Documents/Projects/repo-ciencia-datos/tp-4/clean_data.csv"

In [161]:
def filter_word(word):
    word = word.replace(" ", "")
    word = word.replace("--", "-")
    word = word.upper()
    return word.replace("\\.", "")

In [197]:
co_autores = []
autores = []
lobos_solitarios = []

with open(filename,'r') as f:
    for line in f:
        line = line.split(',')
        line = line[3].split('&')
        cleaned_line = [filter_word(w) for w in line]
        if len(cleaned_line) > 1:
            co_autores.append(cleaned_line)
            for i in cleaned_line:
                autores.append(i)
        else:
            lobos_solitarios.append(cleaned_line)

autores= list(set(autores))

In [198]:
 #6,7, 12
black_list = [
    ')',
    '-p-dedonder',
    '10',
    '14p',
    'a',
    'pages',
    'p',
    'eds.h.-d.doebnere.a.worldsci.',
    'conclusions'
]


In [190]:
for b in black_list:
    print b
    autores.remove(b.upper())

)
-p-dedonder
10
14p
a
pages
p
eds.h.-d.doebnere.a.worldsci.
conclusions


In [199]:
autores_size = len(autores)
print autores_size
#co_autores
adys = np.zeros((autores_size,autores_size))

3684


In [200]:
co_autores_data = pd.DataFrame(data=adys, columns=autores, index=autores)

In [211]:
co_autores_data['AOKI']['AOKI']

0.0

In [ ]:
for A in autores:
    for B in autores:
        for TUPLA in co_autores:
            if(A in TUPLA and B in TUPLA):
                co_autores_data[A][B]=co_autores_data[A][B]+1
                co_autores_data[B][A]=co_autores_data[B][A]+1